# Training the Model to predict if the student talent is suitable to complete the program.

## importing libraries needed.

In [88]:
import pandas as pd 
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


In [89]:
train_df = pd.read_csv("./Dataset/train.csv")
test_df = pd.read_csv("./Dataset/test.csv")

In [105]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Student ID                      818 non-null    object 
 1   Age                             804 non-null    float64
 2   Gender                          818 non-null    object 
 3   Home Region                     817 non-null    object 
 4   Home City                       817 non-null    object 
 5   Program ID                      818 non-null    object 
 6   Program Main Category Code      818 non-null    object 
 7   Program Sub Category Code       703 non-null    object 
 8   Technology Type                 454 non-null    object 
 9   Program Skill Level             618 non-null    object 
 10  Program Presentation Method     818 non-null    object 
 11  Program Start Date              818 non-null    object 
 12  Program End Date                818 

In [90]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6548 entries, 0 to 6547
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Student ID                      6548 non-null   object 
 1   Age                             6456 non-null   float64
 2   Gender                          6548 non-null   object 
 3   Home Region                     6546 non-null   object 
 4   Home City                       6546 non-null   object 
 5   Program ID                      6548 non-null   object 
 6   Program Main Category Code      6548 non-null   object 
 7   Program Sub Category Code       5613 non-null   object 
 8   Technology Type                 3566 non-null   object 
 9   Program Skill Level             4902 non-null   object 
 10  Program Presentation Method     6548 non-null   object 
 11  Program Start Date              6548 non-null   object 
 12  Program End Date                65

In [91]:
train_df.head()

,Student ID,Age,Gender,Home Region,Home City,Program ID,Program Main Category Code,Program Sub Category Code,Technology Type,Program Skill Level,...,Completed Degree,Level of Education,Education Speaciality,College,University Degree Score,University Degree Score System,Employment Status,Job Type,Still Working,Y
0,4f14c50d-162e-4a15-9cf0-ec129c33bcf0,37.0,ذكر,منطقة الرياض,الرياض,453686d8-4023-4506-b2df-fac8b059ac26,PCRF,PCRF,NaN,NaN,...,نعم,البكالوريوس,هندسة حاسب الالي,NaN,2.44,4.0,غير موظف,NaN,NaN,0
1,0599d409-876b-41a5-af05-749ef0e77d32,21.0,ذكر,منطقة عسير,خميس مشيط,cc8e4e42-65d5-4fa1-82f9-6c6c2d508b60,APMR,SWPS,NaN,متوسط,...,نعم,البكالوريوس,الإذاعة والتلفزيون والفيلم,الفنون والعلوم الإنسانية,5.00,5.0,طالب,NaN,NaN,0
2,38a11c0e-4afc-4261-9c64-e94cc0a272fb,24.0,ذكر,منطقة الرياض,الرياض,e006900d-05a9-4c2b-a36f-0ffb9fce44cd,APMR,NaN,NaN,متوسط,...,نعم,البكالوريوس,Information Technology,NaN,3.50,5.0,موظف,NaN,NaN,0
3,1693e85b-f80e-40ce-846f-395ddcece6d3,23.0,ذكر,منطقة الرياض,الرياض,2ec15f6b-233b-428a-b9f5-e40bc8d14cf9,TOSL,TOSL,NaN,NaN,...,نعم,البكالوريوس,حوسبة تطبيقية - (مسار شبكات الحاسب),NaN,3.55,5.0,خريج,NaN,NaN,0
4,98a0e8d0-5f80-4634-afd8-322aa0902863,23.0,ذكر,منطقة الرياض,الرياض,d32da0e9-1aed-48c3-992d-a22f9ccc741e,CAUF,SWPS,تقليدية,متوسط,...,لا,البكالوريوس,نظم المعلومات الحاسوبية,تكنولوجيا الاتصالات والمعلومات,4.00,5.0,NaN,NaN,NaN,0


In [92]:
train_df.describe()

,Age,Program Days,University Degree Score,University Degree Score System,Y
count,6456.000000,6548.000000,6467.000000,6467.000000,6548.000000
mean,26.831165,19.691662,8.224432,9.773929,0.158674
std,5.535967,32.112061,19.120384,21.259962,0.365400
min,18.000000,3.000000,0.000000,4.000000,0.000000
25%,23.000000,5.000000,3.300000,5.000000,0.000000
50%,25.000000,12.000000,4.000000,5.000000,0.000000
75%,29.000000,19.000000,4.510000,5.000000,0.000000
max,57.000000,292.000000,100.000000,100.000000,1.000000


In [93]:
train_df.isnull().sum()

Student ID                           0
Age                                 92
Gender                               0
Home Region                          2
Home City                            2
Program ID                           0
Program Main Category Code           0
Program Sub Category Code          935
Technology Type                   2982
Program Skill Level               1646
Program Presentation Method          0
Program Start Date                   0
Program End Date                     0
Program Days                         0
Completed Degree                     0
Level of Education                  26
Education Speaciality              277
College                           3890
University Degree Score             81
University Degree Score System      81
Employment Status                  566
Job Type                          4567
Still Working                     4567
Y                                    0
dtype: int64

Checking for duplicates

In [94]:
train_df.duplicated().sum()

48

In [95]:
train_df = train_df.drop_duplicates()

In [96]:
train_df.duplicated().sum()

0

In [97]:
train_df["Program Start Date"]

0       2023-05-28
1       2023-04-02
2       2023-07-23
3       2023-07-23
4       2023-04-30
           ...    
6543    2023-04-02
6544    2023-05-14
6545    2023-07-16
6546    2024-01-07
6547    2023-03-19
Name: Program Start Date, Length: 6500, dtype: object

In [82]:
col_to_remove = ["Program Start Date", "Program End Date"]
train_df = train_df.drop(columns=col_to_remove)


In [99]:
for column in train_df.columns:
    if train_df[column].dtype == 'object' or train_df[column].dtype.name == 'category':
        encoder = LabelEncoder()
        train_df[column] = encoder.fit_transform(train_df[column])


In [100]:
X = train_df.drop(columns=['Student ID', 'Y'])  # all columns except the last one as features
y = train_df["Y"]   # the last column as the target variable


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [101]:
model = XGBClassifier()


model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [102]:
# Making predictions
y_pred = model.predict(X_test)

In [103]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.8761538461538462
Precision: 0.6049382716049383
Recall: 0.5025641025641026
F1 Score: 0.5490196078431373
